In [31]:
import simplejson, urllib
import geopandas
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
import googlemaps

from time import sleep as sleep
import datetime as dt
from datetime import timedelta as td
from random import randrange
import json

In [ ]:
class search_handler:
    def __init__(self, api_key):
        self.apartment_list = parse_aparments()

In [57]:
def load_aparments(a_fname="apartments.json", verbose =False):
    f_ap=open(a_fname,)
    start_addresses = json.load(f_ap)
    f_ap.close()
    if verbose:
        print("loaded start addresses:")
        for s in start_addresses:
            print('\t',s)

def load_destinations(dest_fname):
    f = open(dest_fname)
    f\

# a function to parse destinations

# a function to load the api key

# a function to load lat and londitude for an address


import googlemaps
from datetime import datetime

gmaps = googlemaps.Client(key='YOUR KEY')


now = datetime.now()
directions_result = gmaps.directions("18.997739, 72.841280",
                                     "18.880253, 72.945137",
                                     mode="driving",
                                     avoid="ferries",
                                     departure_time=now
                                    )

print(directions_result[0]['legs'][0]['distance']['text'])
print(directions_result[0]['legs'][0]['duration']['text'])

loaded start addresses:
	 2901 Barton Skyway, Austin, TX 78746


In [69]:
from random import randrange
print([randrange(-5,5) for i in range(5)])

[-2, 3, -5, 0, -1]


In [ ]:
a_fname="apartments.json"
dest_f_name='destinations.json'
key_file = 'key.txt'

downsample = 5
cnt = 0
cur_day = dt.datetime.now().weekday()

gmaps=load_gmaps(key_file)

while True:

    for dest in dest_list:
        dest_adr = dest['address']
        print(dest_adr)
        lat_long=get_lat_long(dest_adr)

        dest['lat_long']=lat_long

    for start_adr in start_addresses:               
            print(directions_result[0]['legs'][0]['distance']['text'])
print(directions_result[0]['legs'][0]['duration']['text'])

        #url = "http://maps.googleapis.com/maps/api/distancematrix/json?origins={0}&destinations={1}&mode=driving&language=en-EN&sensor=false".format(str(orig_coord).replace(" ",''),str(dest_coord).replace(" ",''))
        #result= simplejson.load(urllib.request.urlopen(url))
        #print(result)
        #driving_time = result['rows'][0]['elements'][0]['duration']['value']
        #print("drive time from %s to %s is %s"% (start_adr,dest_adr,driving_time))

In [15]:
def add_delta_to_time(t,delta):
    a= (dt.datetime.combine(dt.date(1,1,1),t) + delta)
    a.time()
    return a.time()

print(add_delta_to_time(dt.datetime.now().time(),td(hours = 0)))


13:11:08.636802


In [15]:
def add_delta_to_time(t,delta):
    a= (dt.datetime.combine(dt.date(1,1,1),t) + delta)
    a.time()
    return a.time()

print(add_delta_to_time(dt.datetime.now().time(),td(hours = 0)))


13:11:08.636802
